# RAG (Retrieval-Augmented Generation) based QA system

This notebook implements a Retrieval-Augmented Generation (RAG) based QA system.
We use:

FAISS for document retrieval
Hugging Face Transformers for question answering
Sentence-Transformers for embedding documents



1.   Tutorial: Implementing a basic RAG-based QA system using FAISS for retrieval and Hugging Face Transformers for generation.


2.   Assignment Question: A task to modify/enhance the system within 30 minutes.




1. Install Dependencies

`faiss-cpu:` Fast Approximate Nearest Neighbors (ANN) search for retrieval

`transformers:` Pretrained models for text generation

`datasets:` Load large datasets like Wikipedia

`sentence-transformers:` Convert text into vector embeddings

In [ ]:
!pip install faiss-cpu transformers datasets sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2. Import Libraries



Why these libraries?

`faiss`: Efficient document retrieval

`sentence-transformers:` Converts text to embeddings

`transformers:` Loads Hugging Face models for answering questions

`datasets:` Loads Wikipedia snippets

In [ ]:
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer
from datasets import load_dataset


Load and Embed the Dataset

1. Loads 1000 Wikipedia articles but selects first 500
2. Converts each article into numerical embeddings using all-MiniLM-L6-v2
3. These embeddings allow similarity searches

In [ ]:
# Load sample dataset
dataset = load_dataset("wikipedia", "20220301.simple", split="train[:1000]")  # 1000 articles
docs = dataset["text"][:500]  # Taking 500 docs for efficiency

# Embed using Sentence Transformer
embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedder.encode(docs, convert_to_numpy=True)

# Build FAISS Index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train-00000-of-00001.parquet:   0%|          | 0.00/134M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Build a FAISS Index for Fast Retrieval

Why FAISS?

- FAISS is a fast vector search library
- Uses L2 distance to find the closest documents

Define the Retrieval-Augmented QA Pipeline

`How retrieval works?`
- Encodes the query into an embedding
- Searches for the top k most similar Wikipedia articles
- Returns those relevant documents

In [ ]:
def retrieve_documents(query, k=3):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(query_embedding, k)
    return [docs[i] for i in indices[0]]

# Load HuggingFace Model for Generation
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")


def generate_answer(question):
    retrieved_docs = retrieve_documents(question)
    context = " ".join(retrieved_docs)  # Combine retrieved documents
    input_text = f"Context: {context} Question: {question}"

    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)




tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Retrieve relevant documents using FAISS

- Print the retrieved context (useful for debugging)
- Combine the text into a single input
- Feed it to FLAN-T5 to generate an answer


FLAN-T5 reads the context and answers


In [ ]:
# Test the system
question = "Tell me a science fact"
print(generate_answer(question))


Helium is a noble gas, because it does not regularly mix with other chemicals and form new compounds. It has the lowest boiling point of all the elements. It is the second most common element in the universe, after hydrogen, and has no color or smell. However, helium has a red-orange glow when placed in an electric field.


# **Project**

In [1]:
!pip install beautifulsoup4

In [3]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://devgan.in"
MAIN_URL = f"{BASE_URL}/ipc/"

# Step 1: Get all chapter links
response = requests.get(MAIN_URL)
soup = BeautifulSoup(response.text, "html.parser")

chapters = []
for row in soup.select("table.menu tr"):
    columns = row.find_all("td")
    if len(columns) == 2:
        chapter_number = columns[0].text.strip()
        chapter_title = columns[1].text.strip()
        chapter_link = BASE_URL + columns[1].find("a")["href"]
        chapters.append((chapter_number, chapter_title, chapter_link))

# Step 2: Scrape each chapter's content
for chapter_number, chapter_title, chapter_link in chapters:
    chapter_response = requests.get(chapter_link)
    chapter_soup = BeautifulSoup(chapter_response.text, "html.parser")

    # Extracting the main content - Modify selector if needed
    content_div = chapter_soup.find("div", id="content")

    if content_div:
        chapter_content = content_div.get_text(separator="\n", strip=True)
    else:
        chapter_content = "Content not found."

    print(f"Chapter {chapter_number}: {chapter_title}")
    print(f"URL: {chapter_link}")
    print(chapter_content[:500])  # Print first 500 characters for preview
    print("\n" + "-"*80 + "\n")


Chapter 01: Introduction
URL: https://devgan.in/ipc/chapter_01.php
Preamble
WHEREAS
it is expedient to provide a general Penal Code for India;
It is enacted as follows:
Chapter I – Introduction
Section
1
:- Title and extent of operation of the Code
This Act shall be called the Indian Penal Code, and shall extend to the whole of India except the State of Jammu and Kashmir.
Section
2
:- Punishment of offences committed within India
Every person shall be liable to punishment under this Code and not otherwise for every act or omission contrary to the provisions the

--------------------------------------------------------------------------------

Chapter 02: General Explanations
URL: https://devgan.in/ipc/chapter_02.php
Chapter II – General Explanations
Section
6
:- Definitions in the Code to be understood subject to exceptions
Throughout this Code every definition of an offence, every penal provision and every illustration of every such definition or penal provision, shall be understood s